In [17]:
import numpy as np
import pandas as pd
import yfinance as yf


import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.figure_factory as ff

In [18]:
df = pd.read_csv('asset_returns.csv', names=[i for i in range(83)])
daily_returns = df.fillna(0)
daily_returns.head()

,0,1,2,3,4,5,6,7,8,9,...,73,74,75,76,77,78,79,80,81,82
0,-0.005763,-0.026772,-0.087660,0.026042,-0.003650,-0.074595,-0.057377,-0.084338,-0.084821,0.081127,...,-0.049128,-0.094203,0.000000,-0.083417,0.109715,-0.025634,0.015899,-0.014392,0.042297,-0.010116
1,0.058059,0.113269,0.175373,0.096116,0.076657,0.120327,0.010870,0.102639,0.154634,0.130871,...,-0.029639,0.160000,-0.083081,-0.021158,-0.019817,0.030551,-0.021737,0.003431,0.004292,-0.065693
2,-0.037039,0.035610,-0.026984,-0.077318,-0.144086,-0.012513,-0.053763,-0.031031,-0.013519,-0.117281,...,-0.014870,-0.006897,-0.178512,0.004320,-0.009998,-0.049396,0.000881,-0.068494,-0.036105,0.003123
3,0.055546,-0.002807,0.093801,0.044626,0.089131,0.062302,0.011364,0.067736,0.018844,0.013110,...,0.052459,-0.055556,-0.051674,0.058668,-0.050943,0.096973,0.014222,0.087317,0.034290,0.004675
4,0.037787,0.035890,0.021626,0.163150,0.064524,0.053678,0.024719,0.035753,-0.121480,0.033673,...,0.046823,0.077206,0.036358,0.023387,-0.172145,0.128671,0.077061,0.014368,0.068786,0.062015


In [40]:
def optimal_weights(rets, target_return):
    mean_return = rets.mean().to_numpy().reshape(-1, 1)
    daily_cov = rets.cov().to_numpy()
    e = np.ones(len(mean_return)).reshape(-1, 1)
    result = []
    for r_p in target_return:
        
        top_row = np.hstack((daily_cov, -mean_return, -e))
        middle_row = np.hstack((-mean_return.T, np.zeros((1, 1)), np.zeros((1, 1))))
        bottom_row = np.hstack((-e.T, np.zeros((1, 1)), np.zeros((1, 1))))

        A = np.vstack((top_row, middle_row, bottom_row))
        for row in A:
            print(row)
        b = np.vstack((np.zeros((len(mean_return), 1)), -r_p, -1))

        # Solve the linear system
        try:
            x = np.linalg.solve(A, b)
            result.append(x.flatten())
        except np.linalg.LinAlgError as err:
            print(f"Cannot solve linear system for target return {r_p}: {err}")
            continue

    results_df = pd.DataFrame(result, columns=[f"Asset {i+1}" for i in range(len(mean_return))] + ["Lambda", "Mu"])
    results_df.set_index(target_return, inplace=True)
    return results_df

### Actual average returns, backtesting:

$ \overline{r}^\top w $ - actual average returns

$ w^\top \Sigma w^\top $ - portfolio covariance 

where $w$ denote the optimal portfolio, $\overline{r}$ denote the average return over
the out-of-sample period, and $\Sigma$ denote the out of sample covariance matrix.


In [20]:
def backtesting(optimal_weights, OOS_rets, target_return):
    mean_return_OOS = OOS_rets.mean().to_numpy().reshape(-1, 1)
    daily_cov_OOS = OOS_rets.cov().to_numpy()
    arr = optimal_weights.to_numpy()
    res = []

    for index, row in enumerate(arr):
        weights = row[:-2]
        targ_ret = target_return[index]
        act_ave_return = (mean_return_OOS.T @ weights).item()
        pf_cov = weights.T @ daily_cov_OOS @ weights
        res.append([targ_ret, act_ave_return, pf_cov])
    
    return np.array(res)

# Main

In [41]:
df_weights = optimal_weights(daily_returns, np.linspace(0, 0.1, 21))

[ 1.39795480e-03  3.17478680e-04  4.01009887e-04  5.26611031e-04
  1.15847406e-03  6.35436277e-04  6.21020308e-04  5.76319664e-04
  4.26807328e-04  1.23588877e-03  3.05106235e-04  4.10183994e-04
  6.70277792e-04  3.93627197e-04  2.88731045e-04  3.18718316e-04
  1.34193219e-03  1.46048456e-03  9.67060024e-04  1.61880344e-03
  3.92000884e-04  4.66357832e-04  5.35658486e-04  8.10628274e-04
  4.05725986e-04  1.41302606e-03  4.83298370e-04  1.04235225e-03
  5.71370140e-04  8.11542203e-04  1.31208846e-03  8.87561571e-04
  1.22433426e-03  4.38844898e-04  3.34051670e-04  8.47029889e-04
  3.20341160e-04  8.41591753e-04  1.08570177e-03  3.19890907e-04
  9.71000504e-04  5.14324591e-04  7.42683036e-04  5.78867790e-04
  5.06182271e-04  1.12822825e-03  8.73095005e-04  1.29716333e-03
  7.64379895e-04  7.14234716e-04  5.44935121e-04  8.99706235e-04
  4.70892555e-04  7.47367743e-04  7.63313983e-04  1.12873187e-03
  6.12082075e-04  3.96743467e-04  9.03053740e-04  6.33368631e-04
  4.66539768e-04  4.05191

In [21]:
target_return = np.linspace(0, 0.1, 21)

results_dict = {}
N = len(range(0, daily_returns.shape[0]-100, 12))
for i in range(0, daily_returns.shape[0]-100, 12):
    index = int(i/12)
    start, mid, end = i, i+100, i +112
    daily_returns_IS = daily_returns.iloc[start:mid,:]
    daily_returns_OOS = daily_returns.iloc[mid:end]
    df_weights = optimal_weights(daily_returns_IS, target_return)
    df_act_returns = backtesting(df_weights, daily_returns_OOS, target_return)
    results_dict[index] = pd.DataFrame(df_act_returns, columns=['target_return', 'Actual Average Return', 'Portfolio Covariance']).set_index('target_return')

In [26]:
df_weights

,Asset 1,Asset 2,Asset 3,Asset 4,Asset 5,Asset 6,Asset 7,Asset 8,Asset 9,Asset 10,...,Asset 76,Asset 77,Asset 78,Asset 79,Asset 80,Asset 81,Asset 82,Asset 83,Lambda,Mu
0.000,0.089171,0.069251,0.020385,-0.059977,-0.070582,-0.037232,0.006204,-0.012650,-0.078711,0.135185,...,-0.093648,0.035248,-0.116492,0.095192,-0.155924,0.081752,0.190151,0.077552,-0.001694,1.636153e-05
0.005,0.084132,0.018912,0.016340,-0.151460,-0.163941,-0.088588,-0.166195,0.012762,-0.067034,0.080240,...,-0.061140,0.128793,-0.145449,-0.082572,-0.171259,0.053183,0.297031,0.033268,0.001403,7.890365e-06
0.010,0.079093,-0.031427,0.012295,-0.242942,-0.257300,-0.139945,-0.338595,0.038175,-0.055357,0.025296,...,-0.028632,0.222337,-0.174407,-0.260336,-0.186593,0.024615,0.403911,-0.011017,0.004500,-5.808035e-07
0.015,0.074054,-0.081766,0.008249,-0.334425,-0.350659,-0.191301,-0.510995,0.063588,-0.043680,-0.029648,...,0.003875,0.315882,-0.203364,-0.438100,-0.201928,-0.003953,0.510791,-0.055301,0.007597,-9.051972e-06
0.020,0.069015,-0.132105,0.004204,-0.425908,-0.444018,-0.242657,-0.683394,0.089000,-0.032003,-0.084592,...,0.036383,0.409426,-0.232321,-0.615864,-0.217262,-0.032522,0.617671,-0.099586,0.010695,-1.752314e-05
0.025,0.063976,-0.182444,0.000158,-0.517390,-0.537376,-0.294014,-0.855794,0.114413,-0.020327,-0.139536,...,0.068891,0.502971,-0.261278,-0.793628,-0.232597,-0.061090,0.724551,-0.143870,0.013792,-2.599431e-05
0.030,0.058936,-0.232783,-0.003887,-0.608873,-0.630735,-0.345370,-1.028194,0.139826,-0.008650,-0.194481,...,0.101399,0.596515,-0.290236,-0.971392,-0.247931,-0.089658,0.831431,-0.188155,0.016889,-3.446548e-05
0.035,0.053897,-0.283121,-0.007932,-0.700356,-0.724094,-0.396726,-1.200593,0.165239,0.003027,-0.249425,...,0.133906,0.690059,-0.319193,-1.149156,-0.263266,-0.118226,0.938311,-0.232439,0.019986,-4.293665e-05
0.040,0.048858,-0.333460,-0.011978,-0.791838,-0.817453,-0.448082,-1.372993,0.190651,0.014704,-0.304369,...,0.166414,0.783604,-0.348150,-1.326920,-0.278601,-0.146795,1.045191,-0.276724,0.023084,-5.140781e-05
0.045,0.043819,-0.383799,-0.016023,-0.883321,-0.910812,-0.499439,-1.545393,0.216064,0.026381,-0.359313,...,0.198922,0.877148,-0.377108,-1.504684,-0.293935,-0.175363,1.152071,-0.321008,0.026181,-5.987898e-05


In [22]:
import time

def plot(tret):
    d = []
    e = []
    for key, item in results_dict.items():
        d.append(item['Actual Average Return'].loc[tret])
        e.append(item['Portfolio Covariance'].loc[tret])

    fig = px.line(d)
    fig.add_hline(y=tret, line_dash="dash", line_color="red")
    fig.show()

for t in target_return.tolist():
    plot(t)
